In [134]:
import pandas as pd
import numpy as np
import random
import string
from itertools import combinations

In [156]:
df = pd.DataFrame()
df.index = [f'A{i}' for i in range(1000)]
df['purpose1'] = np.random.rand(1000)
df['purpose2'] = np.random.rand(1000)
df['purpose3'] = np.random.rand(1000)
df['purpose4'] = np.random.rand(1000)
df['purpose5'] = np.random.rand(1000)
df['purpose6'] = np.random.rand(1000)

n=5
bab_list = []
for k in range(5):
    semi_bab_list = []
    for j in range(1000):
        rand_str = ""
        for i in range(n):
            rand_str += str(random.choice(string.ascii_letters))
        semi_bab_list.append(rand_str)
    bab_list.append(semi_bab_list)

for i, val in enumerate(bab_list):
    df[f'bab{i+1}'] = val

df['co2'] = np.random.rand(1000)*100

df['loc1'] = np.random.randint(4, size=(1000))
df['loc2'] = np.random.randint(12, size=(1000))

In [157]:
user = pd.DataFrame()

user['purpose1'] = np.random.rand(1)
user['purpose2'] = np.random.rand(1)
user['purpose3'] = np.random.rand(1)
user['purpose4'] = np.random.rand(1)
user['purpose5'] = np.random.rand(1)
user['purpose6'] = np.random.rand(1)

user['car'] = 1 if np.random.rand(1)[0] > 1/2 else 0
user['bbak'] = 1 if np.random.rand(1)[0] > 1/2 else 0

user['day1'] = np.random.randint(12)
user['day2'] = np.random.randint(12)
user['day3'] = np.random.randint(12)
user['day4'] = np.random.randint(12)

In [158]:
display(df)
display(user)

,purpose1,purpose2,purpose3,purpose4,purpose5,purpose6,bab1,bab2,bab3,bab4,bab5,co2,loc1,loc2
A0,0.275332,0.845806,0.972310,0.468617,0.764582,0.434027,YHngA,uYFFm,zkdbr,uRsmE,ADyRe,24.498972,2,7
A1,0.202960,0.025876,0.646174,0.832832,0.823119,0.639600,KrKrT,VwLlo,TBQhF,Ebgen,jcwoP,4.276693,0,5
A2,0.721448,0.821637,0.584394,0.769236,0.600680,0.892919,FjYRX,WqsWQ,kdFPD,ByaDv,Pligd,55.733124,0,8
A3,0.125202,0.792324,0.219771,0.284858,0.952063,0.895668,aAmOZ,bmyNa,ORQFQ,Ocecb,rShUn,55.323736,1,6
A4,0.703659,0.688924,0.790941,0.239570,0.255237,0.531310,euICN,tHLLu,OrGTU,KuKAK,yFGRI,35.560446,1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A995,0.255114,0.679217,0.377417,0.671295,0.866560,0.449835,XEJwd,pTQvJ,yCAaq,FBpOy,Uilwd,46.126175,2,3
A996,0.363824,0.789917,0.554325,0.327024,0.065786,0.804918,zSZDQ,CiPqz,UwCdY,uxJfe,ovxaM,64.793050,0,8
A997,0.775796,0.834713,0.814343,0.100109,0.707771,0.877080,plFCm,VtiMJ,OCBPz,LbXCq,IcgzH,43.281589,1,4
A998,0.791017,0.520238,0.941525,0.617167,0.576191,0.997162,YWrLQ,XXiET,uwBpr,oAvfk,DCmmw,31.203699,1,7


,purpose1,purpose2,purpose3,purpose4,purpose5,purpose6,car,bbak,day1,day2,day3,day4
0,0.265673,0.596191,0.798494,0.576559,0.049845,0.738386,0,1,1,0,2,2


관광지 위치, 인기도(아직 미구현) 등을 고려하여 1100개의 관광지 중 일부를 필터링

In [54]:
filtered_df = df[:20]

필터링된 관광지 목록 중 관광객이 __'방문할 관광지 수'__ 에 맞게 경우의 수를 구함
- 방문할 관광지 수는 관광객의 car, bbak을 고려하여 정할 예정 (1일당 2개 ~ 4개)
- 방문할 관광지 수를 1일 단위로 할지 여행 단위로 할지는 고려 중  
  
각 관광지 조합에 대해 육각형을 계산하고 후에 유저의 육각형과 유사도 비교

In [168]:
num_tour = (2 + user.car + user.bbak)[0]
num_tour

3

In [55]:
L = []
combination = list(combinations(filtered_df.index, 6))
for comb in combination:
    L.append(filtered_df.loc[comb, ['purpose1','purpose2','purpose3','purpose4','purpose5','purpose6']].sum().values)
    
L = np.array(L)

In [105]:
df_tour = pd.DataFrame(combination)

각 관광지 조합별 관광지 co2 합산

In [ ]:
M = []
for idx in df_tour.index:
    M.append(df.loc[df_tour.loc[idx, :].values, 'co2'].sum())
df_tour['co2'] = M
df_tour

각 관광지 조합별 유저와의 유사도

In [109]:
from numpy import dot
from numpy.linalg import norm
cos_sim = []
for a in L:
    b = user[['purpose1','purpose2','purpose3','purpose4','purpose5','purpose6']].values.reshape(6,)
    cos_sim.append(dot(a, b)/(norm(a)*norm(b)))

In [110]:
df_tour['similarity'] = cos_sim
df_tour

,0,1,2,3,4,5,co2,similarity
0,A0,A1,A2,A3,A4,A5,278.018530,0.877536
1,A0,A1,A2,A3,A4,A6,202.472915,0.840414
2,A0,A1,A2,A3,A4,A7,220.966567,0.855603
3,A0,A1,A2,A3,A4,A8,272.040115,0.852210
4,A0,A1,A2,A3,A4,A9,237.460559,0.827987
...,...,...,...,...,...,...,...,...
38755,A13,A14,A15,A16,A18,A19,326.227725,0.686501
38756,A13,A14,A15,A17,A18,A19,349.240900,0.691502
38757,A13,A14,A16,A17,A18,A19,346.087457,0.700224
38758,A13,A15,A16,A17,A18,A19,289.080651,0.700513


유사도가 높은 순으로 정렬

In [119]:
df_tour_sorted = df_tour.sort_values(by='similarity', ascending=False)
df_tour_sorted

,0,1,2,3,4,5,co2,similarity
20257,A2,A3,A4,A5,A11,A18,315.198301,0.920258
20270,A2,A3,A4,A5,A13,A18,303.117241,0.911469
20264,A2,A3,A4,A5,A12,A18,229.166063,0.908621
3072,A0,A2,A3,A4,A5,A18,243.825472,0.908382
20282,A2,A3,A4,A5,A16,A18,236.292627,0.908108
...,...,...,...,...,...,...,...,...
36635,A6,A9,A10,A13,A14,A19,335.047925,0.502029
10031,A0,A6,A9,A10,A13,A19,282.420521,0.500346
10018,A0,A6,A9,A10,A11,A19,294.501581,0.499955
36599,A6,A9,A10,A11,A14,A19,347.128985,0.499665


유사도가 높으면서 탄소 배출량이 낮은 추천 관광지

In [127]:
recommended_tour = df_tour_sorted[df_tour_sorted['co2']<300].iloc[0, [0,1,2,3,4,5]].values
recommended_tour

array(['A2', 'A3', 'A4', 'A5', 'A12', 'A18'], dtype=object)

각 관광지와 관광지 사이 거리

In [145]:
data = np.random.rand(len(filtered_df.index), len(filtered_df.index))*1000
df_dist = pd.DataFrame(data = data, index = filtered_df.index, columns = filtered_df.index)
df_dist = df_dist+df_dist.T
for i in df_dist.index:
    df_dist.loc[i, i] = 0
df_dist

,A0,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,A17,A18,A19
A0,0.000000,1013.512626,1490.606442,289.663876,587.790976,766.922902,417.338203,1716.897320,1219.548459,917.413828,1283.719819,1252.536785,894.541455,192.643896,1407.825769,539.503949,645.308584,1747.956075,1501.943574,705.679642
A1,1013.512626,0.000000,1794.212880,1396.649990,1488.962784,831.747691,1102.721689,1356.419969,1335.142934,998.542985,834.831959,1369.434739,1505.417470,999.930149,696.329879,995.015190,239.808119,475.815687,680.695544,241.618923
A2,1490.606442,1794.212880,0.000000,339.910850,1201.988367,1373.164810,158.128716,340.842129,1138.445205,1297.230676,499.351332,1369.982393,1192.628370,1561.976323,1433.546821,478.298024,1251.787113,376.264915,754.683161,456.575948
A3,289.663876,1396.649990,339.910850,0.000000,1431.888935,1444.597881,1811.788462,1152.939141,505.491627,1271.950244,937.256945,1214.823471,977.430480,1554.357944,1890.694724,1031.094667,110.945180,411.093746,1022.269033,1049.893886
A4,587.790976,1488.962784,1201.988367,1431.888935,0.000000,508.325529,178.596368,1666.138563,1644.340086,842.287346,339.325227,474.264210,1216.305641,1567.172188,1238.658981,1066.999724,961.089590,767.510292,1516.700855,1163.506235
A5,766.922902,831.747691,1373.164810,1444.597881,508.325529,0.000000,1279.475665,1873.670123,422.143669,1217.530822,870.400230,967.022562,743.619027,1129.109357,1545.962866,1107.177449,452.433767,693.441348,616.363423,673.765157
A6,417.338203,1102.721689,158.128716,1811.788462,178.596368,1279.475665,0.000000,624.410845,358.424759,216.468073,1103.356626,548.683980,551.610645,1220.292380,613.146443,1697.691715,1054.661083,831.609222,243.365922,1754.770553
A7,1716.897320,1356.419969,340.842129,1152.939141,1666.138563,1873.670123,624.410845,0.000000,712.658991,799.289662,1683.486835,1065.305383,1288.431254,566.379179,597.125074,1721.243500,1178.049027,1362.706676,1511.854253,494.788025
A8,1219.548459,1335.142934,1138.445205,505.491627,1644.340086,422.143669,358.424759,712.658991,0.000000,197.611588,943.235584,1251.401108,722.069744,970.183765,996.583894,1706.644703,1082.649723,124.077816,421.700495,915.884428
A9,917.413828,998.542985,1297.230676,1271.950244,842.287346,1217.530822,216.468073,799.289662,197.611588,0.000000,374.685714,1653.329816,1128.858457,951.981123,1682.378821,1160.334558,205.596761,550.194377,1063.326210,1386.654626


In [153]:
# 최단거리 도출

#output : ['A12', 'A5', 'A4', 'A18', 'A2', 'A3']

최단 거리 경로에 따른 밥 추천

In [155]:
min_route = ['A12', 'A5', 'A4', 'A18', 'A2', 'A3']

In [164]:
for i, point in enumerate(min_route):
    if i in [1, 3]: #관광끝내고 밥먹을 곳 선택
        print(df.loc[point, ['bab1', 'bab2', 'bab3', 'bab4', 'bab5']][0]) # 0~4 사이에서 '특정' 기준으로 선택

bzkhw
dWTCs


- 경주 화이팅하기
- 두 관광지 사이의 최단거리 csv 만들기
- co2 threshold 정하는 방법 고려
- 밥타 때 음식점 정하는 방법 고려